In [0]:
%pylab inline

In [0]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd

In [0]:
# Example: load a DSS dataset as a Pandas dataframe
mydataset = dataiku.Dataset("mydataset")
mydataset_df = mydataset.get_dataframe()

In [0]:
import dataiku
import time
client = dataiku.api_client()

p = client.get_project("DKU_CHURN")
d = p.get_dataset("customers_copy")

d.clear()

In [0]:
import dataikuapi
import json

# Source DSS instance
src_host = "http://localhost:11000"
src_api_jey = "ABQFRE9MPCKRX8BAOHU6V5B9F2UFPZ1C"
src_client = dataikuapi.DSSClient(host=src_host, api_key=src_api_jey)

src_project_keys_list = ['DKU_CHURN']

for prj_key in src_project_keys_list:
    
    print('asdfa')
    print('asdfo')
    src_prj = src_client.get_project(prj_key)

    # load datasets list
    dataset_list = set()
    src_datasets = src_prj.list_datasets()
    for dataset in src_datasets:
        
        dssdataset = dataset.to_dataset()
        settings = dssdataset.get_settings()
        raw_settings = settings.get_raw()
        
        try:
            if raw_settings['formatType'] == 'parquet':
                print('here')
                print("Convert dataset " + raw_settings['name'] + " to ORC")
                settings.get_raw()['formatType'] = 'orcfile'
                settings.get_raw()['formatParams'] = { 'compressionMethod': 'SNAPPY', 'serdeClass': 'org.apache.hadoop.hive.ql.io.orc.OrcSerde', 'serdeProperties': {}, 'representsNullFields': False }
                settings.save()
                dssdataset.clear()
                print('dun')
        except:
            print('oopps we did a fucky wucky')

In [0]:
import dataiku
 
def update_trigger_script(trg_dict, new_trg_query_dict):
    if trg_dict['id'] in new_trg_query_dict.keys():
        trg_dict['params']['sql'] = new_trg_query_dict[trg_dict['id']]
    return trg_dict
 
def main():
 
    client = dataiku.api_client() # set up API client
   
    pk = 'DKU_CHURN'
    scn_id = 'what'
    trg_id1 = 'rV7yzlLz'
    trg_id2 = 'cqU0lsQf'
   
    project = client.get_project(pk)
 
    scn_settings = project.get_scenario(scn_id).get_settings()
    trg_lod = scn_settings.get_raw()['triggers']
    print(trg_lod)
    new_trg_query_dict = {trg_id1: 'new script 1', trg_id2: 'new script 2'}
    for x in scn_settings.raw_triggers:
        update_trigger_script(x, new_trg_query_dict)
    scn_settings.save()
   
if __name__ == "__main__":
    main()